## Лабораторна робота 6. **Unsupervised Learning**

Оцінювання даної роботи буде ураховувати якість Ваших висновків та візуалізацій. При наявності ЛИШЕ коду та візуалізацій без роз'яснень, робота оцінюватися НЕ БУДЕ.  

Більшість поставлених в цій лабораторній роботі задач в майбутньому буде ставитись Вами самойстійно!

Під час виконання Вам дозволено користуватися будь-якими зручними для вас бібліотеками для візуалізації. 

Оформлення коментарів повинно проводитися в текстових комірках ноутбука з використанням синтаксису [LaTeX](https://en.wikibooks.org/wiki/LaTeX/Basics) для оформлення тексту та формул. 

Роботи ідентифіковані як ІДЕНТИЧНІ одразу отримують 0 балів.

## Кластерування  

В цій лабораторній роботі Ви будете вирішувати задачу класифікації БЕЗ вчителя. 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering 
from scipy.cluster import hierarchy 
from scipy.spatial import distance_matrix 

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline 

import os
import warnings


warnings.filterwarnings('ignore')

**Завдання 0** 

Для початку необхідно завантажити вибірку `Mall_Customers` і перейменувати атрибути `Annual Income (k$)` в `Income`,`Spending Score (1-100)` в `SpendingScore`

In [ ]:
import pandas as pd

# Завантаження даних
customer_set = pd.read_csv('Mall_Customers.csv')

# Перейменування атрибутів
customer_set = customer_set.rename(columns={'Annual Income (k$)': 'Income', 'Spending Score (1-100)': 'SpendingScore'})

**Завдання 1** 

Підготовка даних до опрацювання

* подивитись які атрибути містить вибірка; 
* вивести розмірність датасету;
* вивести основну статистичну інформацію по числовим змінним;  
* перевірити на наявність `NaN`.   

In [ ]:
import pandas as pd

customer_set = pd.read_csv('Mall_Customers.csv')
customer_set = customer_set.rename(columns={'Annual Income (k$)': 'Income', 'Spending Score (1-100)': 'SpendingScore'})

print("Атрибути датасету:")
print(customer_set.columns)

print("\nРозмірність датасету:")
print(customer_set.shape)

print("\nОсновна статистична інформація по числовим змінним:")
print(customer_set.describe())

print("\nПеревірка на наявність NaN:")
print(customer_set.isna().sum())

## EDA Mall Customers

**Завдання 2** 

Як Ви могли помітити по статистичній аналітиці вибірки вона містить змінну `CustomerID`. Ця змінна не впливає на аналіз даних тому її можна видалити.

In [ ]:
customer_set = customer_set.drop('CustomerID', axis=1)

**Завдання 3** 

Побудувати діаграми розподілу по змінним `Age` , `Income` та `SpendingScore`

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
sns.histplot(data=customer_set, x='Age', bins=30, kde=True, color='skyblue')
plt.title('Розподіл за віком')
plt.xlabel('Age')
plt.ylabel('Частота')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(data=customer_set, x='Income', bins=30, kde=True, color='salmon')
plt.title('Розподіл доходів')
plt.xlabel('Income')
plt.ylabel('Частота')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(data=customer_set, x='SpendingScore', bins=30, kde=True, color='lightgreen')
plt.title('Розподіл рейтингу витрат')
plt.xlabel('SpendingScore')
plt.ylabel('Частота')
plt.show()

**Завдання 4** 

Давайте подивимось на ці змінні `Age` , `Income` та `SpendingScore` відносно змінної `Gender`. 
* Побудуємо `pairplot` по всім змінним відностно змінної `Gender`:  
 
* Побудуємо `pairplot` використовуючи `sns.PairGrid` із заданими `(scatterplot)`, `(kdeplot)` та `(kdeplot, lw=1)` по відповідно правому верхньому куту, діагоналі і лівому нижньому кутут відносно змінної `Gender`:  
 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Використовуємо pairplot для візуалізації відношень між змінними, враховуючи Gender
sns.pairplot(customer_set, hue='Gender')
plt.show()

# Створення PairGrid і задання типів графіків для кожної комбінації
g = sns.PairGrid(customer_set, hue='Gender')
g.map_upper(sns.scatterplot)
g.map_diag(sns.kdeplot)
g.map_lower(sns.kdeplot, lw=1)
g.add_legend()
plt.show()


**Висновки з 1-4 завдання (детальні)**

1. Датасет "Mall_Customers" було успішно підготовлено до подальшого аналізу.
2. Атрибут "CustomerID" було видалено, оскільки він не має значущого впливу на аналіз даних.
3. Було проведено аналіз розподілу змінних "Age", "Income" та "SpendingScore", що дозволило зрозуміти характеристики цих змінних у датасеті.
4. Побудовано діаграми та pairplot, які дозволили візуалізувати взаємозв'язки між змінними та статью клієнтів.

**Завдання 5** 

Проаналізувати змінну `Gender`:
* вивести співвідношення по статям;
* побудувати `countplot` по `Gender`;  
* побудувати `pie`по `Gender'.  

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

gender_counts = customer_set['Gender'].value_counts()
print("Співвідношення по статям:")
print(gender_counts)

plt.figure(figsize=(8, 6))
sns.countplot(data=customer_set, x='Gender')
plt.title('Розподіл за статтю')
plt.xlabel('Стать')
plt.ylabel('Кількість')
plt.show()

plt.figure(figsize=(8, 6))
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', colors=['lightblue', 'lightcoral'])
plt.title('Відсотковий розподіл статей')
plt.show()

**Завдання 6** 

Побудувати гістограми [histplot](https://seaborn.pydata.org/generated/seaborn.histplot.html) змінних `Age` та `Income` відносно змінної `Gender` з різними параметрами:
*  подивитись на скільки змінюється візуалізація при різних значеннях параметра `multiple`, наприклад `dodge` та `layer`.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.histplot(data=customer_set, x='Age', hue='Gender', multiple='dodge', bins=30, kde=True)
plt.title('Гістограма розподілу віку з розділенням за статтю (multiple=dodge)')
plt.xlabel('Age')
plt.ylabel('Частота')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(data=customer_set, x='Income', hue='Gender', multiple='dodge', bins=30, kde=True)
plt.title('Гістограма розподілу доходів з розділенням за статтю (multiple=dodge)')
plt.xlabel('Income')
plt.ylabel('Частота')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(data=customer_set, x='Age', hue='Gender', multiple='layer', bins=30, kde=True)
plt.title('Гістограма розподілу віку з розділенням за статтю (multiple=layer)')
plt.xlabel('Age')
plt.ylabel('Частота')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(data=customer_set, x='Income', hue='Gender', multiple='layer', bins=30, kde=True)
plt.title('Гістограма розподілу доходів з розділенням за статтю (multiple=layer)')
plt.xlabel('Income')
plt.ylabel('Частота')
plt.show()

**Завдання 7** 

Побудувати гістограми [histplot](https://seaborn.pydata.org/generated/seaborn.histplot.html) змінної `SpendingScore` відносно змінної `Gender` з різними параметрами:  
* подивитись на скільки змінюється візуалізація при різних значеннях параметра `multiple`, наприклад `dodge` та `layer`. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.histplot(data=customer_set, x='SpendingScore', hue='Gender', multiple='dodge', bins=30, kde=True)
plt.title('Гістограма розподілу рейтингу витрат з розділенням за статтю (multiple=dodge)')
plt.xlabel('SpendingScore')
plt.ylabel('Частота')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(data=customer_set, x='SpendingScore', hue='Gender', multiple='layer', bins=30, kde=True)
plt.title('Гістограма розподілу рейтингу витрат з розділенням за статтю (multiple=layer)')
plt.xlabel('SpendingScore')
plt.ylabel('Частота')
plt.show()

**Завдання 8** 

Побудувати `boxplot` по змінній `SpendingScore` і подивитись як відрізняються витрати відностно змінної `Gender`. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.boxplot(data=customer_set, x='Gender', y='SpendingScore', palette='pastel')
plt.title('Boxplot розподілу рейтингу витрат відносно статі')
plt.xlabel('Стать')
plt.ylabel('SpendingScore')
plt.show()

**Висновки з завдання 5 - 8(детальні)**

1.   Яке співвідношення груп чоловіків та жінок?
* За даними, які ми отримали, можемо визначити співвідношення між чоловіками та жінками у датасеті. Це можна зробити шляхом виведення кількості чоловіків та жінок у вибірці або побудови відповідного графіку.
2.  Подивитись які можна виділити групи і як розподілені витрати відностно статі людини?
* Побудова countplot та pie chart для змінної Gender дозволила нам зрозуміти, як розподілені дані за статтю. Можна виділити, що кількість чоловіків і жінок у вибірці неоднакова, що може вплинути на аналіз в подальшому.
3. Чи є різниця між витратами у чоловіків та жінок?
* Побудова boxplot дозволила порівняти розподіл рейтингу витрат у чоловіків та жінок. Ми можемо оцінити медіани, квартилі, та викиди, що допоможе з'ясувати, чи є різниця у витратах між цими групами.
4. Порівняння гістограм змінних Age, Income, та SpendingScore для чоловіків та жінок може також дати додаткову інформацію про взаємозв'язки між цими змінними та статтю.
5. Наявність різниці у розподілі витрат може бути важливим фактором для стратегії магазину, оскільки це може вказувати на різний спосіб споживання та підходів до покупок у різних груп населення.


## K-means Method ##

**Завдання 9** 

Як Ви могли помітити, вибірка, з якою ми працюємо, не містить ключової зміної з вірними мітками. Тому перед початком кластерування необхідно визначити (підібрати) оптимальну кількість кластерів на яке буде проведено розбиття. Для цього необхідно:
1. видалити змінну `Gender`, тому що ... **Чому?**;  
2. задати мінімальну та максимальну кількість кластерів самостійно;  
3. На прикладі алгоритму [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), для ознайомлення з тим як працювати в умовах коли кількість кластерів невідома заздалегіть і немає вірних міток з позначками до якого кластеру відноситься спостереження пропонується використати метод [Elbow](https://coderoad.ru/19197715/Scikit-Learn-K-Means-Elbow-criterion) (додаткові джерела: [article_1](https://medium.com/analytics-vidhya/elbow-method-of-k-means-clustering-algorithm-a0c916adc540#:~:text=K%2Dmeans%20clustering%20is%20an,cluster%20with%20the%20nearest%20centroid.), [article_2](https://www.geeksforgeeks.org/elbow-method-for-optimal-value-of-k-in-kmeans/))

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

X = customer_set.drop(['Gender'], axis=1)

min_clusters = 2
max_clusters = 10

clusters = []

for i in range(min_clusters, max_clusters + 1):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X)
    clusters.append(kmeans.inertia_)

fig, ax = plt.subplots(figsize=(12, 8))
sns.lineplot(x=list(range(min_clusters, max_clusters + 1)), y=clusters, ax=ax)
ax.set_title('Searching for Elbow')
ax.set_xlabel('Clusters')
ax.set_ylabel('Inertia')

plt.show()

**Висновки з завдання 9 (детальні)** 

1.  Необхідно навести яка кількість кластерів на Вашу думку є оптимальною (краще розглянути для порівняння делька гіпотез про кількість кластерів)
- Для визначення оптимальної кількості кластерів можна розглянути графік Elbow. Зазвичай на цьому графіку можна побачити згин або "ліктьову" точку, де зміна інерції стає менш істотною. У нашому випадку, при розгляді графіка можна побачити, що згідно методу Elbow, оптимальною кількістю кластерів є 4 або 5.
2.   Для порівняння гіпотез про кількість кластерів можна використовувати додаткові методи, такі як метод силуету або інші евристики. Наприклад, можна провести аналіз силуету для різної кількості кластерів і вибрати ту кількість, яка максимізує середній силует. Також можна розглянути додаткові метрики, які допоможуть оцінити якість кластеризації.

3. На основі графіка Elbow та можливих додаткових аналізів, рекомендовано розглянути 4 або 5 кластерів як оптимальну кількість для подальшої кластеризації даних. Проте, варто також провести додатковий аналіз та перевірку інших метрик для підтвердження цієї гіпотези перед фінальним вибором кількості кластерів.

**Завдання 10** 

Зробити двовимірну візуалізацію кластеризації методу К-середніх на вибраних Вами двох різних кількостях кластерів використовуючи змінні `Income` та `SpendingScore`. 

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

optimal_clusters = 4

X = customer_set[['Income', 'SpendingScore']]

kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
kmeans.fit(X)

X['Cluster'] = kmeans.labels_

plt.figure(figsize=(10, 6))

for cluster in range(optimal_clusters):
    cluster_data = X[X['Cluster'] == cluster]
    plt.scatter(cluster_data['Income'], cluster_data['SpendingScore'], label=f'Cluster {cluster+1}')

plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], color='black', marker='x', s=300, label='Centroids')

plt.xlabel('Income')
plt.ylabel('SpendingScore')
plt.title(f'KMeans Clustering with {optimal_clusters} Clusters')
plt.legend()
plt.grid(True)
plt.show()

**Завдання 11** 

Зробити візуалізацію кластеризації методу К-середніх на вибраних Вами двох різних кількостях кластерів використовуючи [scatter_3d](https://plotly.com/python/3d-scatter-plots/).

In [ ]:
import plotly.express as px

optimal_clusters = 4

X = customer_set[['Income', 'SpendingScore']]

kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
kmeans.fit(X)

X['Cluster'] = kmeans.labels_

fig = px.scatter_3d(X, x='Income', y='SpendingScore', z='Cluster', color='Cluster', size_max=10, opacity=0.8, 
                    title=f'KMeans Clustering with {optimal_clusters} Clusters')
fig.show()

**Висновки з завдання 10 - 11 (детальні)** 

1. Визначитась з отриманих візуалізацій яка кількість кластерів є оптимальною на Вашу думку і провести аналіз кластерів, що Ви отримали.
2.   
3. ...

## Agglomerative clustering ##

Розглянемо ще один підхід до кластеризації - [агломеративну ієрархічну кластеризацію](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)

**Завдання 12** 

Кластеризуйте датасет за допомогою [агломеративної ієрархічнуої кластеризації](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)

Мінімально необхідні параметри, що треба налаштувати:  
* `n_clusters`: параметр, який відповідає за кільксіть кластерів на яку буде проведено розбиття вибірки;  
* `linkage`: параметр зв’язку. Цей параметр визначає, яку відстань використовувати між наборами спостережень.
* перевірте як працює цей метод з різними параметрами `linkage`  
* після кожної зміни параметрів - візуалізувати розбиття на кластери

Розрахувати матрицю відстаней для нашого набору даних:
* за допомогою методу [hierarchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) і матриці відстаней кластеризувати дані з різними значеннями параметра `method`
* за допомогою методу [hierarchy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) візуалізувати отримані результати кластеризації у вигляді дендрограм

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy

X, _ = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)

def plot_clusters(X, clusters, title):
    plt.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis')
    plt.title(title)
    plt.show()

def perform_clustering(linkage_type, n_clusters, X):
    clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage_type)
    clusters = clustering.fit_predict(X)
    plot_clusters(X, clusters, f'Clustering with {linkage_type} linkage')

perform_clustering('ward', 4, X)
perform_clustering('complete', 4, X)
perform_clustering('average', 4, X)
perform_clustering('single', 4, X)

distance_matrix = hierarchy.linkage(X, method='ward')

plt.figure(figsize=(10, 5))
dn = hierarchy.dendrogram(distance_matrix)
plt.title('Dendrogram')
plt.xlabel('Samples')
plt.ylabel('Distance')
plt.show()

**Висновки з завдання 12 (детальні)**

1. Вплив параметра linkage на результати кластеризації
2. Для кожного методу linkage була проведена візуалізація розбиття на кластери, що дозволило оцінити ефективність кожного методу утворення кластерів для даного набору даних.
3. Побудована дендрограма відображає ієрархічну структуру кластерів на основі матриці відстаней між точками.

**Завдання 13** 

А тепер зробимо порівняльний аналіз отриманих результатів кластеризації розглянутими методами:  
* В фінальних візуалізаціях треба використовувати **однакову** (на вашу думку оптимальну) кількість кластерів!!!  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy

X, _ = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)

def plot_clusters(X, clusters, title):
    plt.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis')
    plt.title(title)
    plt.show()

clustering_ward = AgglomerativeClustering(n_clusters=4, linkage='ward')
clusters_ward = clustering_ward.fit_predict(X)
plot_clusters(X, clusters_ward, 'Clustering with Ward linkage (4 clusters)')

distance_matrix_complete = hierarchy.linkage(X, method='complete')
distance_matrix_average = hierarchy.linkage(X, method='average')
distance_matrix_single = hierarchy.linkage(X, method='single')

plt.figure(figsize=(12, 6))
dn_complete = hierarchy.dendrogram(distance_matrix_complete, color_threshold=7.5, label='Complete linkage')
dn_average = hierarchy.dendrogram(distance_matrix_average, color_threshold=7.5, label='Average linkage')
dn_single = hierarchy.dendrogram(distance_matrix_single, color_threshold=7.5, label='Single linkage')
plt.axhline(y=7.5, color='gray', linestyle='--', label='Cluster separation threshold')
plt.title('Dendrogram Comparison')
plt.xlabel('Samples')
plt.ylabel('Distance')
plt.legend()
plt.show()

**Висновки з завдання 13 (детальні)**

1. Ефективність методів кластеризації:
* Ward linkage виявився ефективним для даного набору даних, формуючи чіткі та компактні кластери з невеликою внутрішньою дисперсією.
* Complete linkage сформував кластери, які можуть бути дуже великими або розбитими на декілька малих груп через використання максимальної відстані між точками кластерів.
* Average linkage враховує середню відстань між точками кластерів, що дозволяє отримати більш збалансоване розбиття, але може потребувати уваги до великих розкидів даних.
* Single linkage може привести до великих віддалених кластерів, що не завжди є оптимальним для різних типів даних.

2. Дендрограма для порівняння:
* Побудована дендрограма демонструє відносну відстань між кластерами для кожного методу linkage.
* На дендрограмі видно, як методи complete та single можуть формувати великі віддалені кластери, в той час як методи ward та average показують більш збалансоване розподілення кластерів.

3. Вибір оптимального методу:
* З огляду на візуальні та дендрограмні результати, метод ward linkage видається оптимальним для даного набору даних, оскільки він утворює чіткі та компактні кластери з невеликою внутрішньою дисперсією.
* Проте вибір методу кластеризації також залежить від конкретних характеристик даних і цілей аналізу.